In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sc
import numpy as np
import xtrack as xt
import xcoll as xc
import xobjects as xo
import xpart as xp
from pathlib import Path
import lossmaps as lm
import cernlayoutdb as layout

 If you are on SWAN and did not expect this warning, then restart and load the 'NXCals Pro' software stack.


# Making line without apertures of acc models

Useless because I already had it ...

In [2]:
#Loading line with apertures from acc models
line_2 = xt.Line.from_json('injection_lines/injection_thick_el_aperture.json')

#Removing aperture elements
elems_to_remove = line_2.get_elements_of_type([xt.LimitRectEllipse, xt.LimitEllipse, xt.LimitRect, xt.LimitRacetrack, xt.LimitPolygon])[1]

#Making new line without the aperture elements
new_names = [name for name in line_2.element_names if name not in elems_to_remove]
new_elements_dict = {name: line_2.element_dict[name] for name in new_names}
line = xt.Line(elements = new_elements_dict, particle_ref=line_2.particle_ref)

#Saving line to json
line.to_json('injection_lines/injection_thick_no_aperture_self.json')

Loading line from dict:   0%|          | 0/4571 [00:00<?, ?it/s]

Done loading line from dict.           


In [3]:
#line = xt.Line.from_json('injection_lines/injection_thick_no_aperture_self.json')
line = xt.Line.from_json('injection_lines/injection_thick_no_ap_with_deferred_expressions.json')

Loading line from dict:   0%|          | 0/4426 [00:00<?, ?it/s]

Done loading line from dict.           


# Finding unique elements to look for apertures

In [4]:
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Bend)[1]]))
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Quadrupole)[1]]))
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Sextupole)[1]]))
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Octupole)[1]]))
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Multipole)[1]]))
print(np.unique([name[:4] for name in line_2.get_elements_of_type(xt.Cavity)[1]]))

['mba.' 'mbb.']
['lqsa' 'qd.1' 'qd.2' 'qd.3' 'qd.4' 'qd.5' 'qd.6' 'qda.' 'qe.1' 'qe.3'
 'qe.5' 'qe.6' 'qecd' 'qf.1' 'qf.2' 'qf.3' 'qf.4' 'qf.5' 'qf.6' 'qfa.'
 'qms.']
['lsd.' 'lse.' 'lsen' 'lsf.']
['lod.' 'loe.' 'loen' 'lof.']
['adkc' 'adks' 'mdh.' 'mdha' 'mdhb' 'mdhd' 'mdhw' 'mdph' 'mdsh' 'mdv.'
 'mdva' 'mdvw' 'mkdh' 'mkdv' 'mke.' 'mkp.' 'mkpa' 'mkpc' 'mkqh' 'mkqv'
 'mplh' 'mplv' 'mpnh' 'mpsh' 'mpsv' 'mse.' 'mst.' 'zkha' 'zkv.' 'zs.2']
['acfc' 'acl.' 'actc']


# Adding apertures of elements from LDB (approximation)

In [4]:
db = layout.DB()

In [5]:
db.get_apertures_by_element('qf.10010'.upper(), version="LS3 1.7", machine='SPS')['ELEM_TYPE_APERTURE_ID']

select ELEM_NAME,TYPE_NAME,ELEM_APERTURE_ALIAS,ELEM_TYPE_APERTURE_ID,ELEMENT_APERTURE_ID,ELEM_APERTURE_X,ELEM_APERTURE_Y,ELEM_APERTURE_Z,ELEM_ABS_ROT_B,ELEM_ABS_ROT_A,ELEM_ABS_ROT_C from ELEMENT_APERTURES_VERSION_V where ELEM_NAME LIKE 'QF.10010' AND VERSION_TAG='LS3 1.7' AND MACHINE_CODE='SPS'


array([13003604, 54913763])

In [6]:
tab = layout.Table.from_ldb("APERTURE_PROFILES")

In [7]:
aperture_info_list = db.get_apertures(machine='SPS', version='EYETS 2024-2025') #There is a first aperture at -0.095, which wll give a negative drift

In [8]:
ap_inf_names = [aperture_info_list[i][1].lower() for i in range(len(aperture_info_list))]

In [9]:
l_uniques = {'Bend' : np.unique([name[:4] for name in line.get_elements_of_type(xt.Bend)[1]]),
             'Quadrupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Quadrupole)[1]]),
             'Sextupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Sextupole)[1]]),
             'Octupole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Octupole)[1]]),
             'Multipole': np.unique([name[:4] for name in line.get_elements_of_type(xt.Multipole)[1]]),
             'Cavity': np.unique([name[:4] for name in line.get_elements_of_type(xt.Cavity)[1]])}

In [9]:
# Initialize a dictionary to store the results
verification_results = {}

# Loop through the elements in l_uniques
for element_type, unique_names in l_uniques.items():
    verification_results[element_type] = {}
    for name in unique_names:
        # Check if any name in ap_inf_names starts with the current unique name
        is_matched = any(ap_name.startswith(name.lower()) for ap_name in ap_inf_names)
        verification_results[element_type][name] = is_matched

# Print or process the verification results
for element_type, results in verification_results.items():
    print(f"Verification results for {element_type}:")
    for name, is_found in results.items():
        status = "Found" if is_found else "Not Found"
        print(f"  {name}: {status}")


Verification results for Bend:
  mba.: Found
  mbb.: Found
Verification results for Quadrupole:
  lqsa: Not Found
  qd.1: Found
  qd.2: Found
  qd.3: Found
  qd.4: Found
  qd.5: Found
  qd.6: Found
  qda.: Found
  qe.1: Not Found
  qe.3: Not Found
  qe.5: Not Found
  qe.6: Not Found
  qecd: Not Found
  qf.1: Found
  qf.2: Found
  qf.3: Found
  qf.4: Found
  qf.5: Found
  qf.6: Found
  qfa.: Not Found
  qms.: Found
Verification results for Sextupole:
  lsd.: Found
  lse.: Found
  lsen: Found
  lsf.: Found
Verification results for Octupole:
  lod.: Not Found
  loe.: Found
  loen: Found
  lof.: Found
Verification results for Multipole:
  adkc: Not Found
  adks: Not Found
  mdh.: Found
  mdha: Found
  mdhb: Not Found
  mdhd: Not Found
  mdhw: Not Found
  mdph: Found
  mdsh: Found
  mdv.: Found
  mdva: Found
  mdvw: Not Found
  mkdh: Not Found
  mkdv: Not Found
  mke.: Not Found
  mkp.: Not Found
  mkpa: Not Found
  mkpc: Not Found
  mkqh: Found
  mkqv: Not Found
  mplh: Not Found
  mplv: F

In [10]:
# Initialize a dictionary to store the counts
counts = {}

# Loop through the elements in l_uniques
for element_type, unique_names in l_uniques.items():
    counts[element_type] = {}
    for name in unique_names:
        # Count how many names in ap_inf_names start with the current unique name
        match_count = sum(1 for ap_name in ap_inf_names if ap_name.startswith(name.lower()))
        counts[element_type][name] = match_count

# Print or process the counts
for element_type, element_counts in counts.items():
    print(f"Counts for {element_type}:")
    for name, count in element_counts.items():
        print(f"  {name}: {count} matches")


Counts for Bend:
  mba.: 720 matches
  mbb.: 768 matches
Counts for Quadrupole:
  lqsa: 0 matches
  qd.1: 34 matches
  qd.2: 32 matches
  qd.3: 36 matches
  qd.4: 32 matches
  qd.5: 36 matches
  qd.6: 32 matches
  qda.: 14 matches
  qe.1: 0 matches
  qe.3: 0 matches
  qe.5: 0 matches
  qe.6: 0 matches
  qecd: 0 matches
  qf.1: 36 matches
  qf.2: 32 matches
  qf.3: 36 matches
  qf.4: 34 matches
  qf.5: 32 matches
  qf.6: 34 matches
  qfa.: 0 matches
  qms.: 8 matches
Counts for Sextupole:
  lsd.: 108 matches
  lse.: 18 matches
  lsen: 2 matches
  lsf.: 108 matches
Counts for Octupole:
  lod.: 0 matches
  loe.: 20 matches
  loen: 12 matches
  lof.: 48 matches
Counts for Multipole:
  adkc: 0 matches
  adks: 0 matches
  mdh.: 204 matches
  mdha: 10 matches
  mdhb: 0 matches
  mdhd: 0 matches
  mdhw: 0 matches
  mdph: 4 matches
  mdsh: 2 matches
  mdv.: 202 matches
  mdva: 14 matches
  mdvw: 0 matches
  mkdh: 0 matches
  mkdv: 0 matches
  mke.: 0 matches
  mkp.: 0 matches
  mkpa: 0 matches


In [11]:
counts_unique_elems = {}

for element_type, element_counts in counts.items():
    for name, count in element_counts.items():
        counts_unique_elems[str(name)] = count

In [12]:
# Loop through all element names in the line
missing_ap_ldb = []

for name in line.element_names:
    # Check if the name is not in ap_inf_names
    if name.lower() not in ap_inf_names:
        # Check if it starts with any pattern in l_uniques
        if any(name.startswith(pattern) for patterns in l_uniques.values() for pattern in patterns):
            print(name)
            missing_ap_ldb.append(name)

lod.10302
lod.10702
qe.11402
mkqv.11679
mdhw.11732
mdhw.11737
mdhw.11738
mdhw.11754
mkpa.11931
mkpa.11936
mkpc.11952
mkp.11955
lod.12702
lqsa.12902
lod.13302
lod.13502
lod.20302
lod.20702
mplh.21431
qfa.21610
zs.21633
zs.21639
zs.21655
zs.21671
zs.21676
mst.21774
mst.21779
mst.21794
qfa.21810
mse.21832
mse.21837
mse.21852
mse.21857
mse.21872
mplh.21995
mplh.22195
lod.22702
lqsa.22902
lod.23302
lod.23502
qe.30202
qecd.30602
lod.30702
qecd.31402
adkcv.32171
adkcv.32172
adksv.32173
lod.32702
lqsa.32902
lod.33302
lod.33502
lod.40302
lod.40702
mke.41631
mke.41634
mke.41637
mke.41651
mplh.41658
qfa.41810
mse.41837
mse.41852
mse.41856
mse.41871
mse.41876
mse.41891
mplh.41994
lod.42702
lqsa.42902
lod.43302
lod.43502
qe.50202
lod.50302
lod.50702
qe.51402
qfa.51610
mdhw.51634
mdhw.51637
mdvw.51694
mdvw.51696
mkdvb.51698
mkdva.51731
mkdvb.51735
mkdha.51751
mkdha.51754
mkdhb.51757
qfa.51810
mdhd.51832
lod.52702
lqsa.52902
lod.53302
qecd.53402
lod.53502
qe.60302
qe.60402
qe.60502
qe.60602
lod.60702

In [13]:
for name in missing_ap_ldb:
    if counts_unique_elems[name[:4]] != 0:
        print(name)

Nothing is printed in the cell above, so it means that there is no element of a certain category that needs to be set. 

Maybe I will have to do it based on the element type though (ex: no qe value at all, so analyse the other quadrupole apertures)

# Verifiying if all unique identifiers have same apertures

In [11]:
dico_types = {}
for elem in aperture_info_list:
    name = elem[1].lower()
    if any(name.startswith(pattern) for patterns in l_uniques.values() for pattern in patterns):
        if name[:4] not in dico_types:
            dico_types[name[:4]] = [elem[2]]
        else:
            if elem[2] not in dico_types[name[:4]]:
                dico_types[name[:4]].append(elem[2])

In [12]:
dico_types

{'qf.1': ['SPQF___'],
 'mba.': ['SPMBA__'],
 'mbb.': ['SPMBB__'],
 'lsd.': ['SPLSD__'],
 'mdv.': ['SPMDV__'],
 'qd.1': ['SPQD___'],
 'lsf.': ['SPLSF__'],
 'mdh.': ['SPMDH__'],
 'loe.': ['SPLOE__'],
 'lse.': ['SPLSE__'],
 'lof.': ['SPLOF__'],
 'qms.': ['SPQMS__'],
 'mkqh': ['SPMKQH_'],
 'mdph': ['SPMDPH_'],
 'mdva': ['SPMDVA_'],
 'qda.': ['SPQDA__'],
 'mdsh': ['SPMDSH_'],
 'loen': ['SPLOEN_'],
 'qf.2': ['SPQF___'],
 'qd.2': ['SPQD___'],
 'mpsh': ['SPMPSH_'],
 'mpsv': ['SPMPSV_'],
 'mdha': ['SPMDHA_'],
 'mpnh': ['SPMPNH_'],
 'zkha': ['SPZKHA_'],
 'zkv.': ['SPZKV__'],
 'qf.3': ['SPQF___'],
 'qd.3': ['SPQD___'],
 'actc': ['SPACTCSE',
  'SPACTCSF',
  'SPACTCSG',
  'SPACTCSH',
  'SPACTCSI',
  'SPACTCSJ'],
 'acl.': ['SPACL__'],
 'qf.4': ['SPQF___'],
 'qd.4': ['SPQD___'],
 'mplv': ['SPMPLV_'],
 'lsen': ['SPLSEN_'],
 'qf.5': ['SPQF___'],
 'qd.5': ['SPQD___'],
 'qf.6': ['SPQF___'],
 'qd.6': ['SPQD___']}

In [13]:
aperture_data_per_unique = {}

for key in dico_types:
    if len(dico_types[key])== 1:
        print(key)
        print(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(' ')
        aperture_data_per_unique[key] = {'aper_type' : str(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_x' : float(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'max_y' : float(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'a' : float(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0]),
                                       'b' : float(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]])[0])}
        
        #a = np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]])
        #b = np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==dico_types[key][0]])
        #c = np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==dico_types[key][0]])
        #d = np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==dico_types[key][0]])
        
        #if len(a) > 1 or len(b) > 1 or len(c) > 1 or len(d) > 1:
            #print(key)

qf.1
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
mba.
[0.0712]
[0.01725]
[0.0712]
[0.01725]
['RECTANGLE']
 
mbb.
[0.0597]
[0.02425]
[0.0597]
[0.02425]
['RECTANGLE']
 
lsd.
[0.0415]
[0.0415]
[0.0415]
[0.0415]
['CIRCLE']
 
mdv.
[0.0415]
[0.0415]
[0.0415]
[0.0415]
['CIRCLE']
 
qd.1
[0.0415]
[0.0415]
[0.0415]
[0.0415]
['CIRCLE']
 
lsf.
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
mdh.
[0.076]
[0.01765]
[0.152]
[0.0353]
['RECTANGLE']
 
loe.
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
lse.
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
lof.
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
qms.
[0.076]
[0.019]
[0.076]
[0.019]
['ELLIPSE']
 
mkqh
[0.0575]
[0.01615]
[0.0575]
[0.01615]
['ELLIPSE']
 
mdph
[0.0775]
[0.0415]
[0.0775]
[0.0415]
['ELLIPSE']
 
mdva
[0.071]
[0.071]
[0.071]
[0.071]
['CIRCLE']
 
qda.
[0.0755]
[0.0455]
[0.0755]
[0.0455]
['ELLIPSE']
 
mdsh
[0.065]
[0.0345]
[0.065]
[0.0345]
['RECTANGLE']
 
loen
[0.076]
[0.01915]
[0.076]
[0.01915]
['ELLIPSE']
 
qf.2
[0.076

In [17]:
aperture_data_per_unique

{'qf.1': {'aper_type': 'ELLIPSE',
  'max_x': 0.076,
  'max_y': 0.01915,
  'a': 0.076,
  'b': 0.01915},
 'mba.': {'aper_type': 'RECTANGLE',
  'max_x': 0.0712,
  'max_y': 0.01725,
  'a': 0.0712,
  'b': 0.01725},
 'mbb.': {'aper_type': 'RECTANGLE',
  'max_x': 0.0597,
  'max_y': 0.02425,
  'a': 0.0597,
  'b': 0.02425},
 'lsd.': {'aper_type': 'CIRCLE',
  'max_x': 0.0415,
  'max_y': 0.0415,
  'a': 0.0415,
  'b': 0.0415},
 'mdv.': {'aper_type': 'CIRCLE',
  'max_x': 0.0415,
  'max_y': 0.0415,
  'a': 0.0415,
  'b': 0.0415},
 'qd.1': {'aper_type': 'CIRCLE',
  'max_x': 0.0415,
  'max_y': 0.0415,
  'a': 0.0415,
  'b': 0.0415},
 'lsf.': {'aper_type': 'ELLIPSE',
  'max_x': 0.076,
  'max_y': 0.01915,
  'a': 0.076,
  'b': 0.01915},
 'mdh.': {'aper_type': 'RECTANGLE',
  'max_x': 0.076,
  'max_y': 0.01765,
  'a': 0.152,
  'b': 0.0353},
 'loe.': {'aper_type': 'ELLIPSE',
  'max_x': 0.076,
  'max_y': 0.01915,
  'a': 0.076,
  'b': 0.01915},
 'lse.': {'aper_type': 'ELLIPSE',
  'max_x': 0.076,
  'max_y': 0.01

In [18]:
for key in dico_types:
    if len(dico_types[key])== 1:
        print(key)
        print(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==dico_types[key][0]]))
        print(' ')

qf.1
['ELLIPSE']
 
mba.
['RECTANGLE']
 
mbb.
['RECTANGLE']
 
lsd.
['CIRCLE']
 
mdv.
['CIRCLE']
 
qd.1
['CIRCLE']
 
lsf.
['ELLIPSE']
 
mdh.
['RECTANGLE']
 
loe.
['ELLIPSE']
 
lse.
['ELLIPSE']
 
lof.
['ELLIPSE']
 
qms.
['ELLIPSE']
 
mkqh
['ELLIPSE']
 
mdph
['ELLIPSE']
 
mdva
['CIRCLE']
 
qda.
['ELLIPSE']
 
mdsh
['RECTANGLE']
 
loen
['ELLIPSE']
 
qf.2
['ELLIPSE']
 
qd.2
['CIRCLE']
 
mpsh
['RECTANGLE']
 
mpsv
['CIRCLE']
 
mdha
['ELLIPSE']
 
mpnh
['ELLIPSE']
 
zkha
['RECTANGLE']
 
zkv.
['RECTANGLE']
 
qf.3
['ELLIPSE']
 
qd.3
['CIRCLE']
 
acl.
['CIRCLE']
 
qf.4
['ELLIPSE']
 
qd.4
['CIRCLE']
 
mplv
['CIRCLE']
 
lsen
['ELLIPSE']
 
qf.5
['ELLIPSE']
 
qd.5
['CIRCLE']
 
qf.6
['ELLIPSE']
 
qd.6
['CIRCLE']
 


No unique identifier that has only one type has different apertures, so that is good

In [23]:
dico_types['actc']

['SPACTCSE', 'SPACTCSF', 'SPACTCSG', 'SPACTCSH', 'SPACTCSI', 'SPACTCSJ']

In [24]:
for type in dico_types['actc']:
    print(type)
    print(np.unique([elem[4][1][0] for elem in aperture_info_list if elem[2]==type]))
    print(np.unique([elem[4][1][1] for elem in aperture_info_list if elem[2]==type]))
    print(np.unique([elem[4][1][2] for elem in aperture_info_list if elem[2]==type]))
    print(np.unique([elem[4][1][3] for elem in aperture_info_list if elem[2]==type]))
    print(np.unique([elem[4][0] for elem in aperture_info_list if elem[2]==type]))
    print(' ')

SPACTCSE
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 
SPACTCSF
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 
SPACTCSG
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 
SPACTCSH
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 
SPACTCSI
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 
SPACTCSJ
[0.065]
[0.065]
[0.065]
[0.065]
['CIRCLE']
 


In [14]:
aperture_data_per_unique['actc'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : 0.065,
                                       'max_y' : 0.065,
                                       'a' : 0.065,
                                       'b' : 0.065}

Even actc, that has different types, has the same apertures all the time

Aperture data has been collected in dictionary aperture_data_per_unique

In [15]:
for key in aperture_data_per_unique:
    aper_type = aperture_data_per_unique[key]['aper_type']
    max_x = aperture_data_per_unique[key]['max_x']
    max_y = aperture_data_per_unique[key]['max_y']
    a = aperture_data_per_unique[key]['a']
    b = aperture_data_per_unique[key]['b']
    
    if aper_type == 'CIRCLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'ELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'RECTANGLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRect(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y)
    elif aper_type == 'RECTELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRectEllipse(max_x=max_x, max_y=max_y, a=a, b=b)
    elif aper_type == 'RACETRACK':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRacetrack(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y, a=a, b=b)

# Inserting elements

In [28]:
insert_names = [{f'{name}_aper_upstream': index, f'{name}_aper_downstream': index+1} for index, name in enumerate(line.element_names) if name.lower()[:4] in aperture_data_per_unique]
insert_names = {kk:vv for dct in insert_names for kk, vv in dct.items()}
idxs = list(insert_names.values())
names = list(insert_names.keys())
max_length = max(max(map(len, line.element_names)),max(map(len, names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, idxs, names)

insert_elements = [{f'{name}_aper_upstream': aperture_data_per_unique[name[:4]]['xt_elem'].copy(), f'{name.lower()}_aper_downstream': aperture_data_per_unique[name[:4]]['xt_elem'].copy()} for name in line.element_names if name.lower()[:4] in aperture_data_per_unique]
insert_elements = {kk:vv for dct in insert_elements for kk, vv in dct.items()}

line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_elements}

In [29]:
line.check_aperture()

Checking aperture:   0%|          | 0/7241 [00:00<?, ?it/s]

Done checking aperture.           
66 thin elements miss associated aperture (upstream):
['mkqv.11679',
 'mdhw.11732',
 'mdhw.11737',
 'mdhw.11738',
 'mdhw.11754',
 'mkpa.11931',
 'mkpa.11936',
 'mkpc.11952',
 'mkp.11955',
 'mplh.21431',
 'zs.21633',
 'zs.21639',
 'zs.21655',
 'zs.21671',
 'zs.21676',
 'mst.21774',
 'mst.21779',
 'mst.21794',
 'mse.21832',
 'mse.21837',
 'mse.21852',
 'mse.21857',
 'mse.21872',
 'mplh.21995',
 'mplh.22195',
 'adkcv.32171',
 'adkcv.32172',
 'adksv.32173',
 'mke.41631',
 'mke.41634',
 'mke.41637',
 'mke.41651',
 'mplh.41658',
 'mse.41837',
 'mse.41852',
 'mse.41856',
 'mse.41871',
 'mse.41876',
 'mse.41891',
 'mplh.41994',
 'mdhw.51634',
 'mdhw.51637',
 'mdvw.51694',
 'mdvw.51696',
 'mkdvb.51698',
 'mkdva.51731',
 'mkdvb.51735',
 'mkdha.51751',
 'mkdha.51754',
 'mkdhb.51757',
 'mdhd.51832',
 'mke.61631',
 'mke.61634',
 'mke.61637',
 'mplh.61655',
 'acfca.61739',
 'acfca.61740',
 'mst.61779',
 'mst.61794',
 'mdhb.61805',
 'mse.61832',
 'mse.61837',
 'mse.

,s,element_type,name,isthick,isreplica,parent_name,iscollective,element,s_start,s_center,s_end,is_aperture,i_aperture_upstream,s_aperture_upstream,i_aperture_downstream,s_aperture_downstream,misses_aperture_upstream,misses_aperture_downstream,has_aperture_problem
0,0.0000,Marker,sps$start,False,False,None,False,Marker(),0.0000,0.0000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
1,0.0000,Marker,begi.10010,False,False,None,False,Marker(),0.0000,0.0000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
2,0.0000,LimitEllipse,qf.10010_aper_upstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",0.0000,0.0000,0.0000,True,NaN,NaN,NaN,NaN,False,False,False
3,0.0000,Quadrupole,qf.10010,True,False,None,False,"Quadrupole(k1=0.0116, k1s=0, length=3.08, num_...",0.0000,1.5425,3.0850,False,2.0,0.0,4.0,3.085,False,False,False
4,3.0850,LimitEllipse,qf.10010_aper_downstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",3.0850,3.0850,3.0850,True,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7239,6911.1338,Drift,bph.63608,True,False,None,False,Drift(length=0.275),6911.1338,6911.2713,6911.4088,False,NaN,NaN,NaN,NaN,False,False,False
7240,6911.4088,Drift,drift_1880,True,False,None,False,Drift(length=0.095),6911.4088,6911.4563,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
7241,6911.5038,Marker,end.10010,False,False,None,False,Marker(),6911.5038,6911.5038,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
7242,6911.5038,Marker,sps$end,False,False,None,False,Marker(),6911.5038,6911.5038,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False


# Missing apertures

Bjorn found them in /afs/cern.ch/eng/sps/2015/aperture

In [30]:
for name in missing_ap_ldb:
    if 'acfc' in name:
        print(name)

acfca.61739
acfca.61740


In [31]:
for key in counts_unique_elems:
    if counts_unique_elems[key] == 0:
        c = 0
        for name in missing_ap_ldb:
            if name[:4] == key:
                c += 1
        print(f'{key} : {c}')

lqsa : 6
qe.1 : 1
qe.3 : 1
qe.5 : 2
qe.6 : 4
qecd : 3
qfa. : 6
lod. : 28
adkc : 2
adks : 1
mdhb : 1
mdhd : 1
mdhw : 6
mdvw : 2
mkdh : 3
mkdv : 3
mke. : 7
mkp. : 1
mkpa : 2
mkpc : 1
mkqv : 1
mplh : 7
mse. : 16
mst. : 5
zs.2 : 5
acfc : 2


In [32]:
np.sqrt(0.0935**2/2)

np.float64(0.0661144840409422)

In [16]:
aperture_data_per_unique['lqsa'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.08,
                                       'max_y' : 0.08,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified to a rectangle'}

aperture_data_per_unique['qe.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qecd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0661,
                                       'max_y' : 0.0661,
                                       'a' : None,
                                       'b' : None,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['qfa.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.05378,
                                       'b' : 0.05378,
                                       'Note' : 'Simplified and approximated'}

aperture_data_per_unique['lod.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415,
                                       'Note' : 'From old file, no CAD in LDB'}

aperture_data_per_unique['adkcv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['adksv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1,
                                       'Note' : 'no CAD in LDB'}

aperture_data_per_unique['mdhb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.36,
                                       'max_y' : 0.118,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.32,
                                       'max_y' : 0.09,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdvw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.2,
                                       'max_y' : 0.2,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdh'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103,
                                       'Note' : 'mkdhb, no mkdha and even b is weird drawing'}

aperture_data_per_unique['mkdv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mke.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkp'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.103,
                                       'b' : 0.103}

aperture_data_per_unique['mkqv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.08102,
                                       'b' : 0.08102}

aperture_data_per_unique['mplh'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.05,
                                       'max_y' : 0.38617,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mse.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339}

aperture_data_per_unique['mst.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1339,
                                       'b' : 0.1339,
                                       'Note' : 'No cad but its supposed to be one tank'}

aperture_data_per_unique['zs.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.10755,
                                       'b' : 0.10755}


aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

In [34]:
107.56/2

53.78

I'm doing it manually by grep, assuming they are all the same type. But when grepping for lqsa for example, I only have one match and no install. So there is clearly some incomplete stuff...

For qe.6, needs to be done manually: qe.1,3,5 is 0.076, 0.01915, same for qe.60402, 60602, but for 60502 and 60302 it's 0.0415, 0.0415

aperture_data_per_unique['lqsa'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415}

aperture_data_per_unique['qe.1'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qe.3'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qe.5'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qe.60402'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qe.60602'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qe.60302'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415}

aperture_data_per_unique['qe.60502'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415}

aperture_data_per_unique['qecd'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.076,
                                        'b' : 0.01915}

aperture_data_per_unique['qfa.1'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.0755,
                                        'b' : 0.0455}

aperture_data_per_unique['qfa.2'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.1065,
                                        'b' : 0.0203}

aperture_data_per_unique['qfa.4'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.1065,
                                        'b' : 0.0203}


aperture_data_per_unique['qfa.6'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.1065,
                                        'b' : 0.0203}

aperture_data_per_unique['qfa.5'] = {'aper_type' : 'ELLIPSE',
                                        'max_x' : None,
                                        'max_y' : None,
                                        'a' : 0.1065,
                                        'b' : 0.0203}

aperture_data_per_unique['lod.'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415}

aperture_data_per_unique['mdhb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.128,
                                       'max_y' : 0.053,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhd'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.083,
                                       'max_y' : 0.0625,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdhw'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.076,
                                       'max_y' : 0.01935,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mdvw'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.0415,
                                       'b' : 0.0415}

aperture_data_per_unique['mkdha'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.048,
                                       'max_y' : 0.028,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdhb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0525,
                                       'max_y' : 0.030,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdva'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0375,
                                       'max_y' : 0.028,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkdvb'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0415,
                                       'max_y' : 0.028,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkp.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.07,
                                       'max_y' : 0.027,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mke.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.0675,
                                       'max_y' : 0.016,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkpa'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.05,
                                       'max_y' : 0.0305,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkpc'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.05,
                                       'max_y' : 0.0305,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mkqv'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.051,
                                       'max_y' : 0.028,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mplh'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.076,
                                       'max_y' : 0.01765,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mse.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.091,
                                       'max_y' : 0.078,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['mst.'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.054,
                                       'max_y' : 0.078,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['zs.2'] = {'aper_type' : 'RECTANGLE',
                                       'max_x' : 0.068,
                                       'max_y' : 0.023,
                                       'a' : None,
                                       'b' : None}

aperture_data_per_unique['adkcv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

aperture_data_per_unique['adkcv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

aperture_data_per_unique['adksv'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

aperture_data_per_unique['acfca'] = {'aper_type' : 'CIRCLE',
                                       'max_x' : None,
                                       'max_y' : None,
                                       'a' : 0.1,
                                       'b' : 0.1}

In [35]:
for name in line.element_names:
    if name[:4]=='mdhw':
        print(name)

mdhw.11732
mdhw.11737
mdhw.11738
mdhw.11754
mdhw.51634
mdhw.51637


mdhw.51634: rectangle, 0.076, 0.01935

mdhw.51637: rectangle, 0.076, 0.01765

Others do not exist, and they are completely random anyway. For now, I will just put the biggest one for all of them to have an approx...

In [36]:
for name in line.element_names:
    if name[:4]=='mdvw':
        print(name)

mdvw.51694
mdvw.51696


mdvw.51694
mdvw.51696

they do not exist in the file. One of them has rect 0.04, 0.0415 and the two others have ellipse 0.0415, 0.0415, so I will take these values.

In [37]:
for name in line.element_names:
    if name[:4]=='mkdh':
        print(name)

mkdha.51751
mkdha.51754
mkdhb.51757


In [38]:
for name in line.element_names:
    if name[:4]=='mke.':
        print(name)

mke.41631
mke.41634
mke.41637
mke.41651
mke.61631
mke.61634
mke.61637


mke. does not exist, only mkel and mkes, I took the most restricting one. mkp. not either, but MKP is defined and different than mkpa and mkpc. All three exist

In [39]:
for name in line.element_names:
    if name[:4]=='mse.':
        print(name)

mse.21832
mse.21837
mse.21852
mse.21857
mse.21872
mse.41837
mse.41852
mse.41856
mse.41871
mse.41876
mse.41891
mse.61832
mse.61837
mse.61852
mse.61857
mse.61872


# Making approximated line

In [22]:
#line = xt.Line.from_json('injection_lines/injection_thick_no_aperture_self.json')
line = xt.Line.from_json('sps_lhc_q20_rf.json')

Loading line from dict:   0%|          | 0/14648 [00:00<?, ?it/s]

Done loading line from dict.           


In [17]:
slice_factor = 4
line.slice_thick_elements(slicing_strategies=[xt.Strategy(slicing=xt.Teapot(1)),
                                              xt.Strategy(slicing=xt.Teapot(2), element_type=xt.Bend),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), element_type=xt.Quadrupole),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxa.*'),
                                              xt.Strategy(slicing=xt.Teapot(32*slice_factor), name='mqxb.*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqxb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mbrs\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwa\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4), name='mqwb\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqy\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqm\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqmc\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(4*slice_factor), name='mqml\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtlh\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqtli\\..*'),
                                              xt.Strategy(slicing=xt.Teapot(2*slice_factor), name='mqt\\..*'),
                                              xt.Strategy(slicing=None, element_type=xt.Solenoid),
                                              xt.Strategy(slicing=None, name='qe\\..*'), 
                                              xt.Strategy(slicing=None, name='qfa\\..*'),
                                              xt.Strategy(slicing=None, name='qecd\\..*')])

Slicing line:   0%|          | 0/4426 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010_entry',
  'drift_qf.10010..0',
  'qf.10010..0',
  'drift_qf.10010..1',
  'qf.10010..1',
  'drift_qf.10010..2',
  'qf.10010..2',
  'drift_qf.10010..3',
  'qf.10010..3',
  'drift_qf.10010..4',
  'qf.10010..4',
  'drift_qf.10010..5',
  'qf.10010..5',
  'drift_qf.10010..6',
  'qf.10010..6',
  'drift_qf.10010..7',
  'qf.10010..7',
  'drift_qf.10010..8',
  'qf.10010_exit'],
 'drift_0': ['drift_0'],
 'mba.10030': ['mba.10030_entry',
  'mba.10030..entry_map',
  'drift_mba.10030..0',
  'mba.10030..0',
  'drift_mba.10030..1',
  'mba.10030..1',
  'drift_mba.10030..2',
  'mba.10030..exit_map',
  'mba.10030_exit'],
 'drift_1': ['drift_1'],
 'mba.10050': ['mba.10050_entry',
  'mba.10050..entry_map',
  'drift_mba.10050..0',
  'mba.10050..0',
  'drift_mba.10050..1',
  'mba.10050..1',
  'drift_mba.10050..2',
  'mba.10050..exit_map',
  'mba.10050_exit'],
 'drift_2': ['drift_2'],
 'mbb.10070': ['mbb.10070_entry',
  'mbb.1

In [18]:
for key in aperture_data_per_unique:
    aper_type = aperture_data_per_unique[key]['aper_type']
    max_x = aperture_data_per_unique[key]['max_x']
    max_y = aperture_data_per_unique[key]['max_y']
    a = aperture_data_per_unique[key]['a']
    b = aperture_data_per_unique[key]['b']
    
    if aper_type == 'CIRCLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'ELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitEllipse(a=a, b=b)
    elif aper_type == 'RECTANGLE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRect(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y)
    elif aper_type == 'RECTELLIPSE':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRectEllipse(max_x=max_x, max_y=max_y, a=a, b=b)
    elif aper_type == 'RACETRACK':
        aperture_data_per_unique[key]['xt_elem'] = xt.LimitRacetrack(min_x=-max_x, max_x=max_x, min_y=-max_y, max_y=max_y, a=a, b=b)

In [19]:
# Update insert_names to handle variable-length keys
insert_names = [
    {f'{name}_aper_upstream': index, f'{name}_aper_downstream': index + 1}
    for index, name in enumerate(line.element_names)
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_names = {kk: vv for dct in insert_names for kk, vv in dct.items()}
idxs = list(insert_names.values())
names = list(insert_names.keys())
max_length = max(max(map(len, line.element_names)), max(map(len, names)))
element_names = np.array(line.element_names, dtype=f'<U{max_length}')
names = np.array(names, dtype=f'<U{max_length}')
element_names = np.insert(element_names, idxs, names)

# Update insert_elements to handle variable-length keys
insert_elements = [
    {
        f'{name}_aper_upstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy(),
        f'{name}_aper_downstream': aperture_data_per_unique[next(key for key in aperture_data_per_unique if name.lower().startswith(key))]['xt_elem'].copy()
    }
    for name in line.element_names
    if any(name.lower().startswith(key) for key in aperture_data_per_unique)
]
insert_elements = {kk: vv for dct in insert_elements for kk, vv in dct.items()}

# Update line's element names and element dictionary
line.element_names = element_names.tolist()
line.element_dict = {**line.element_dict, **insert_elements}

In [20]:
line.twiss()

TwissTable: 30213 rows, 62 cols
name                                       s             x            px             y ...
sps$start                                  0             0             0             0
begi.10010                                 0             0             0             0
qf.10010_entry_aper_upstream               0             0             0             0
qf.10010_entry                             0             0             0             0
qf.10010_entry_aper_downstream             0             0             0             0
drift_qf.10010..0                          0             0             0             0
qf.10010..0_aper_upstream           0.171389             0             0             0
qf.10010..0                         0.171389             0             0             0
qf.10010..0_aper_downstream         0.171389             0             0             0
drift_qf.10010..1                   0.171389             0             0             0
...
mdh

In [21]:
line.check_aperture()

Checking aperture:   0%|          | 0/30210 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


,s,element_type,name,isthick,isreplica,parent_name,iscollective,element,s_start,s_center,s_end,is_aperture,i_aperture_upstream,s_aperture_upstream,i_aperture_downstream,s_aperture_downstream,misses_aperture_upstream,misses_aperture_downstream,has_aperture_problem
0,0.0000,Marker,sps$start,False,False,None,False,Marker(),0.0000,0.0000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
1,0.0000,Marker,begi.10010,False,False,None,False,Marker(),0.0000,0.0000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
2,0.0000,LimitEllipse,qf.10010_entry_aper_upstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",0.0000,0.0000,0.0000,True,NaN,NaN,NaN,NaN,False,False,False
3,0.0000,Marker,qf.10010_entry,False,False,None,False,Marker(),0.0000,0.0000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
4,0.0000,LimitEllipse,qf.10010_entry_aper_downstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",0.0000,0.0000,0.0000,True,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30208,6911.1338,Drift,bph.63608,True,False,None,False,Drift(length=0.275),6911.1338,6911.2713,6911.4088,False,NaN,NaN,NaN,NaN,False,False,False
30209,6911.4088,Drift,drift_1879,True,False,None,False,Drift(length=0.095),6911.4088,6911.4563,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
30210,6911.5038,Marker,end.10010,False,False,None,False,Marker(),6911.5038,6911.5038,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
30211,6911.5038,Marker,sps$end,False,False,None,False,Marker(),6911.5038,6911.5038,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False


In [22]:
line.to_json('injection_thin_approx_ap_with_deferred_exppressions.json')

In [31]:
line.check_aperture()

Checking aperture:   0%|          | 0/28455 [00:00<?, ?it/s]

Done checking aperture.           
0 thin elements miss associated aperture (upstream):
[]
0 thick elements miss associated aperture (upstream or downstream):
[]


,s,element_type,name,isthick,isreplica,parent_name,iscollective,element,s_start,s_center,s_end,is_aperture,i_aperture_upstream,s_aperture_upstream,i_aperture_downstream,s_aperture_downstream,misses_aperture_upstream,misses_aperture_downstream,has_aperture_problem
0,0.0000,Marker,sps$start,False,False,None,False,Marker(),0.0000,0.00000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
1,0.0000,Marker,begi.10010,False,False,None,False,Marker(),0.0000,0.00000,0.0000,False,NaN,NaN,NaN,NaN,False,False,False
2,0.0000,Drift,drift_0,True,False,None,False,Drift(length=0.308),0.0000,0.15425,0.3085,False,NaN,NaN,NaN,NaN,False,False,False
3,0.3085,LimitEllipse,qf.10010..1_aper_upstream,False,False,None,False,"LimitEllipse(a=0.076, b=0.0192)",0.3085,0.30850,0.3085,True,NaN,NaN,NaN,NaN,False,False,False
4,0.3085,Multipole,qf.10010..1,False,False,None,False,"Multipole(_order=np.int64(1), inv_factorial_or...",0.3085,0.30850,0.3085,False,3.0,0.3085,5.0,0.3085,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28454,6911.2713,Drift,bph.63608,True,False,None,False,Drift(length=0),6911.2713,6911.27130,6911.2713,False,NaN,NaN,NaN,NaN,False,False,False
28455,6911.2713,Drift,drift_7321,True,False,None,False,Drift(length=0.233),6911.2713,6911.38755,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
28456,6911.5038,Marker,end.10010,False,False,None,False,Marker(),6911.5038,6911.50380,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
28457,6911.5038,Marker,sps$end,False,False,None,False,Marker(),6911.5038,6911.50380,6911.5038,False,NaN,NaN,NaN,NaN,False,False,False
